In [ ]:
!pip3 install rank_bm25 -q

In [1]:
!pip3 install torch
!pip3 install transformers

  Using cached https://files.pythonhosted.org/packages/13/70/54e9fb010fe1547bc4774716f11ececb81ae5b306c05f090f4461ee13205/torch-1.5.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3d/fc/4763e5f17ac6e7e7d55f377cde859ca1c5d5ac624441ab45315bc578aa9e/numpy-1.18.3-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 573kB 2.1MB/s ta 0:00:01
    100% |████████████████████████████████| 1.0MB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 6.7MB/s eta 0:00:01
    100% |████████████████████████████████| 3.7MB 438kB/s eta 0:00:01
    100% |████████████████████████████████| 686kB 222kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/3d/fc/4763e5f17ac6e7e7d55f377cde859ca1c5d5ac624441ab45315bc578aa9e/numpy-1.18.3-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 61kB 5.5MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 6.1MB/s eta 0:00:01
    100% |

In [12]:
import torch
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

BERT_SQUAD = 'bert-large-uncased-whole-word-masking-finetuned-squad'

model = BertForQuestionAnswering.from_pretrained(BERT_SQUAD)
tokenizer = BertTokenizer.from_pretrained(BERT_SQUAD)


model = model.to(torch_device)
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [13]:
def answer_question(question, context):
    # anser question given question and context
    encoded_dict = tokenizer.encode_plus(
                        question, context,
                        add_special_tokens = True,
                        max_length = 256,
                        pad_to_max_length = True,
                        return_tensors = 'pt'
                   )
    
    input_ids = encoded_dict['input_ids'].to(torch_device)
    token_type_ids = encoded_dict['token_type_ids'].to(torch_device)
    
    start_scores, end_scores = model(input_ids, token_type_ids=token_type_ids)

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    
    answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])
    answer = answer.replace('[CLS]', '')
    return answer

In [14]:
context='Kerala is a state on the southwestern Malabar Coast of India. It was formed on 1 November 1956, following the passage of the States Reorganisation Act, by combining Malayalam-speaking regions of the erstwhile states of Travancore-Cochin and Madras. Spread over 38,863 km2 (15,005 sq mi), Kerala is the twenty-third largest Indian state by area. It is bordered by Karnataka to the north and northeast, Tamil Nadu to the east and south, and the Lakshadweep Sea[11] to the west. With 33,387,677 inhabitants as per the 2011 Census, Kerala is the thirteenth-largest Indian state by population. It is divided into 14 districts with the capital being Thiruvananthapuram. Malayalam is the most widely spoken language and is also the official language of the state.[12]  The Chera Dynasty was the first prominent kingdom based in Kerala. The Ay kingdom in the deep south and the Ezhimala kingdom in the north formed the other kingdoms in the early years of the Common Era (CE or AD). The region had been a prominent spice exporter since 3000 BCE. The region\'s prominence in trade was noted in the works of Pliny as well as the Periplus around 100 CE. In the 15th century, the spice trade attracted Portuguese traders to Kerala, and paved the way for European colonisation of India. At the time of Indian independence movement in the early 20th century, there were two major princely states in Kerala-Travancore State and the Kingdom of Cochin. They united to form the state of Thiru-Kochi in 1949. The Malabar region, in the northern part of Kerala, had been a part of the Madras province of British India, which later became a part of the Madras State post-independence. After the States Reorganisation Act, 1956, the modern-day state of Kerala was formed by merging the Malabar district of Madras State (excluding Gudalur taluk of Nilgiris district, Topslip, the Attappadi Forest east of Anakatti), the state of Thiru-Kochi (excluding four southern taluks of Kanyakumari district, Shenkottai and Tenkasi taluks), and the taluk of Kasaragod (now Kasaragod District) in South Canara (Tulunad) which was a part of Madras State.  The economy of Kerala is the 11th-largest state economy in India with ₹8.76 trillion (US$120 billion) in gross domestic product and a per capita GDP of ₹199,000 (US$2,800).[3][13] Kerala has the lowest positive population growth rate in India, 3.44%; the highest Human Development Index (HDI), 0.784 in 2018 (0.712 in 2015); the highest literacy rate, 93.91% in the 2011 census; the highest life expectancy, 77 years; and the highest sex ratio, 1,084 women per 1,000 men. The state has witnessed significant emigration, especially to the Arab states of the Persian Gulf during the Gulf Boom of the 1970s and early 1980s, and its economy depends significantly on remittances from a large Malayali expatriate community. Hinduism is practised by more than half of the population, followed by Islam and Christianity. The culture is a synthesis of Aryan, Dravidian, Arab, and European cultures,[14] developed over millennia, under influences from other parts of India and abroad.  The production of pepper and natural rubber contributes significantly to the total national output. In the agricultural sector, coconut, tea, coffee, cashew and spices are important. The state\'s coastline extends for 595 kilometres (370 mi), and around 1.1 million people in the state are dependent on the fishery industry which contributes 3% to the state\'s income. The state has the highest media exposure in India with newspapers publishing in nine languages, mainly English and Malayalam. Kerala is one of the prominent tourist destinations of India, with backwaters, hill stations, beaches, Ayurvedic tourism and tropical greenery as its major attractions.  The name Kerala has an uncertain etymology. One folk etymology derives Kerala from the Malayalam word kera \'coconut tree\' and alam \'land\'; thus, \'land of coconuts\',[15] which is a nickname for the state used by locals due to the abundance of coconut trees.[16] The word Kerala is first recorded as Keralaputra in a 3rd-century BCE rock inscription left by the Maurya emperor Ashoka (274–237 BCE), one of his edicts pertaining to welfare.[17] The inscription refers to the local ruler as Keralaputra (Sanskrit for \'son of Kerala\' or \'son of Chera[s]\'). This contradicts the theory that kera is from \'coconut tree\'.[18] At that time, one of three states in the region was called Cheralam in Classical Tamil: Chera and Kera are variants of the same word.[19] The word Cheral refers to the oldest known dynasty of Kerala kings and is derived from the Proto-Tamil-Malayalam word for \'lake\'.[20]  The earliest Sanskrit text to mention Kerala as Cherapadha is the late Vedic text Aitareya Aranyaka. Kerala is also mentioned in the Ramayana and the Mahabharata, the two Hindu epics.[21] The Skanda Purana mentions the ecclesiastical office of the Thachudaya Kaimal who is referred to as Manikkam Keralar, synonymous with the deity of the Koodalmanikyam temple.[22][23] Keralam may stem from the Classical Tamil cherive-alam \'declivity of a hill or a mountain slope\'[24] or chera alam \'land of the Cheras\'. The Greco-Roman trade map Periplus Maris Erythraei refers to Keralaputra as Celobotra.[25]  According to the Tamil classic Purananuru, the Chera king Senkuttuvan conquered the lands between Kanyakumari and the Himalayas.[26] Lacking worthy enemies, he besieged the sea by throwing his spear into it.[26][27] According to the 17th century Malayalam work Keralolpathi, the lands of Kerala were recovered from the sea by the axe-wielding warrior sage Parasurama, the sixth avatar of Vishnu (hence, Kerala is also called Parasurama Kshetram \'The Land of Parasurama\'[28]). Parasurama threw his axe across the sea, and the water receded as far as it reached. According to legend, this new area of land extended from Gokarna to Kanyakumari.[29] The land which rose from sea was filled with salt and unsuitable for habitation; so Parasurama invoked the Snake King Vasuki, who spat holy poison and converted the soil into fertile lush green land. Out of respect, Vasuki and all snakes were appointed as protectors and guardians of the land. P. T. Srinivasa Iyengar theorised, that Senguttuvan may have been inspired by the Parasurama legend, which was brought by early Aryan settlers.[30]  Another much earlier Puranic character associated with Kerala is Mahabali, an Asura and a prototypical just king, who ruled the earth from Kerala. He won the war against the Devas, driving them into exile. The Devas pleaded before Lord Vishnu, who took his fifth incarnation as Vamana and pushed Mahabali down to netherworld to placate the Devas. There is a belief that, once a year during the Onam festival, Mahabali returns to Kerala.[31] The Matsya Purana, among the oldest of the 18 Puranas,[32][33] uses the Malaya Mountains of Kerala (and Tamil Nadu) as the setting for the story of Matsya, the first incarnation of Vishnu, and Manu, the first man and the king of the region.[34][35]  A substantial portion of Kerala may have been under the sea in ancient times. Marine fossils have been found in an area near Changanacherry, thus supporting the hypothesis.[36] Pre-historical archaeological findings include dolmens of the Neolithic era in the Marayur area of the Idukki district. They are locally known as "muniyara", derived from muni (hermit or sage) and ara (dolmen).[37] Rock engravings in the Edakkal Caves, in Wayanad date back to the Neolithic era around 6000 BCE.[38][39] Archaeological studies have identified Mesolithic, Neolithic and Megalithic sites in Kerala.[40] The studies point to the development of ancient Kerala society and its culture beginning from the Paleolithic Age, through the Mesolithic, Neolithic and Megalithic Ages.[41] Foreign cultural contacts have assisted this cultural formation;[42] historians suggest a possible relationship with Indus Valley Civilization during the late Bronze Age and early Iron Age.[43]  Kerala has been a major spice exporter since 3000 BCE, according to Sumerian records and it is still referred to as the "Garden of Spices" or as the "Spice Garden of India".[44][45]:79 Kerala\'s spices attracted ancient Babylonians, Assyrians and Egyptians to the Malabar Coast in the 3rd and 2nd millennia BCE. Phoenicians established trade with Kerala during this period.[46] The Land of Keralaputra was one of the four independent kingdoms in southern India during Ashoka\'s time, the others being Chola, Pandya, and Satiyaputra.[47] Scholars hold that Keralaputra is an alternate name of the Cheras, the first dominant dynasty based in Kerala.[48][49] These territories once shared a common language and culture, within an area known as Tamilakam.[50] Along with the Ay kingdom in the south and the Ezhimala kingdom in the north, the Cheras formed the ruling kingdoms of Kerala in the early years of the Common Era (CE).[51] It is noted in Sangam literature that the Chera king Uthiyan Cheralathan ruled most of modern Kerala from his capital in Kuttanad,[52][53] and controlled the port of Muziris, but its southern tip was in the kingdom of Pandyas,[54] which had a trading port sometimes identified in ancient Western sources as Nelcynda (or Neacyndi) in Quilon.[55] The lesser known Ays and Mushikas kingdoms lay to the south and north of the Chera regions respectively.[56][57]  In the last centuries BCE the coast became important to the Greeks and Romans for its spices, especially black pepper. The Cheras had trading links with China, West Asia, Egypt, Greece, and the Roman Empire.[58] In foreign-trade circles the region was known as Male or Malabar.[59] Muziris, Berkarai, and Nelcynda were among the principal ports at that time.[60] The value of Rome\'s annual trade with the region was estimated at around 50,000,000 sesterces;[61] contemporary Sangam literature describes Roman ships coming to Muziris in Kerala, laden with gold to exchange for pepper. One of the earliest western traders to use the monsoon winds to reach Kerala was Eudoxus of Cyzicus, around 118 or 166 BCE, under the patronage of Ptolemy VIII, king of the Hellenistic Ptolemaic dynasty in Egypt. Roman establishments in the port cities of the region, such as a temple of Augustus and barracks for garrisoned Roman soldiers, are marked in the Tabula Peutingeriana, the only surviving map of the Roman cursus publicus.[62][63]  Merchants from West Asia and Southern Europe established coastal posts and settlements in Kerala.[64] The Israeli (Jewish) connection with Kerala started in 573 BCE.[65][66][67] Arabs also had trade links with Kerala, starting before the 4th century BCE, as Herodotus (484–413 BCE) noted that goods brought by Arabs from Kerala were sold to the Israelis [Hebrew (Jews)] at Eden.[60] In the 4th century, the Knanaya or Southist Christians also migrated from Persia and lived alongside the early Syriac Christian community known as the St. Thomas Christians who trace their origins to the evangelistic activity of Thomas the Apostle in the 1st century.[68][69] Mappila was an honorific title that had been assigned to respected visitors from abroad; Israelite (Jewish), Syrian Christian, and Muslim immigration account for later names of the respective communities: Juda Mappilas, Nasrani Mappilas and Muslim Mappilas.[70][71] The earliest Saint Thomas Christian Churches,[72] Cheraman Jumu\'ah Masjid (traditionally dated to "629 CE" by the Mappilas)—regarded as "the first mosque of India"[73]—and Paradesi Synagogue (1568 CE)—the oldest active synagogue in the Commonwealth of Nations[74]—were built in Kerala.[75]  Namboothiri Brahmins migrated to Kerala during the early Middle Ages. Apart from introducing the caste system, they also changed the socio-economic life of the people by commissioning new religious centres.[76] A second Chera Kingdom (c. 800–1102), also known as Kulasekhara dynasty of Mahodayapuram (present-day Kodungallur), was established by Kulasekhara Varman, which ruled over a territory comprising the whole of modern Kerala and a smaller part of modern Tamil Nadu. During the early part of the Kulasekara period, the southern region from Nagercoil to Thiruvalla was ruled by Ay kings, who lost their power in the 10th century, making the region a part of the Kulasekara empire.[77][78] Under Kulasekhara rule, Kerala witnessed a developing period of art, literature, trade and the Bhakti movement of Hinduism.[79] A Keralite identity, distinct from the Tamils, became linguistically separate during this period around the seventh century.[80] For local administration, the empire was divided into provinces under the rule of Naduvazhis, with each province comprising a number of Desams under the control of chieftains, called as Desavazhis.[79]  The inhibitions, caused by a series of Chera-Chola wars in the 11th century, resulted in the decline of foreign trade in Kerala ports. In addition, Portuguese invasions in the 15th century caused two major religions, Buddhism and Jainism, to disappear from the land. It is known that the Menons in the Malabar region of Kerala were originally strong believers of Jainism.[81] The social system became fractured with divisions on caste lines.[82] Finally, the Kulasekhara dynasty was subjugated in 1102 by the combined attack of Later Pandyas and Later Cholas.[77] However, in the 14th century, Ravi Varma Kulashekhara (1299–1314) of the southern Venad kingdom was able to establish a short-lived supremacy over southern India. After his death, in the absence of a strong central power, the state was divided into thirty small warring principalities; the most powerful of them were the kingdom of Samuthiri in the north, Venad in the south and Kochi in the middle. In the 18th Century, Travancore King Sree Anizham Thirunal Marthanda Varma annexed all the kingdoms up to Northern Kerala through military conquests, resulting in the rise of Travancore to pre-eminence in Kerala.The Kochi ruler sued for peace with Anizham Thirunal and Malabar came under direct British rule until India became independent.[83][84]  The maritime spice trade monopoly in the Indian Ocean (Indu Maha Samundr) stayed with the Arabs during the High and Late Middle Ages. However, the dominance of Middle East traders was challenged in the European Age of Discovery. After Vasco Da Gama\'s arrival in Kappad Kozhikode in 1498, the Portuguese began to dominate eastern shipping, and the spice-trade in particular.[85][86][87] They established a trading center at Tangasseri in Quilon during 1502 as per the invitation of the then Queen of Quilon to start spices trade from there.[88] The Zamorin of Kozhikode permitted the new visitors to trade with his subjects such that Portuguese trade in Kozhikode prospered with the establishment of a factory and a fort. However, Portuguese attacks on Arab properties in his jurisdiction provoked the Zamorin and led to conflicts between them.  The Portuguese took advantage of the rivalry between the Zamorin and the King of Kochi allied with Kochi. When Francisco de Almeida was appointed as Viceroy of Portuguese India in 1505, his headquarters was established at Fort Kochi (Fort Emmanuel) rather than in Kozhikode. During his reign, the Portuguese managed to dominate relations with Kochi and established a few fortresses on the Malabar Coast.[89] Fort St Angelo or St. Angelo Fort was built at Kannur in 1505 and Fort St Thomas was built at Kollam(Quilon) in 1518 by the Portuguese.[90] However, the Portuguese suffered setbacks from attacks by Zamorin forces in Malabar region; especially from naval attacks under the leadership of Kozhikode admirals known as Kunjali Marakkars, which compelled them to seek a treaty. An insurrection at the Port of Quilon between the Arabs and the Portuguese led to the end of the Portuguese era in Quilon. In 1571, the Portuguese were defeated by the Zamorin forces in the battle at Chaliyam Fort.[91]  The Portuguese were ousted by the Dutch East India Company, who during the conflicts between the Kozhikode and the Kochi, gained control of the trade.[92] They lost to Dutch at Quilon after 1661 and later, the Portuguese left south-western coast.[93] The Dutch in turn were weakened by constant battles with Marthanda Varma of the Travancore Royal Family, and were defeated at the Battle of Colachel in 1741.[94] An agreement, known as "Treaty of Mavelikkara", was signed by the Dutch and Travancore in 1753, according to which the Dutch were compelled to detach from all political involvement in the region.[95][96][97] Marthanda Varma annexed northern kingdoms through military conquests, resulting in the rise of Travancore to a position of preeminence in Kerala.[98]  In 1766, Hyder Ali, the ruler of Mysore invaded northern Kerala.[99] His son and successor, Tipu Sultan, launched campaigns against the expanding British East India Company, resulting in two of the four Anglo-Mysore Wars.[100][101] Tipu ultimately ceded the Malabar District and South Kanara to the company in the 1790s; both were annexed to the Madras Presidency of British India in 1792.[102][103][104] The company forged tributary alliances with Kochi in 1791 and Travancore in 1795.[105]  By the end of 18th century, the whole of Kerala fell under the control of the British, either administered directly or under suzerainty.[106] There were major revolts in Kerala during the independence movement in the 20th century; most notable among them is the 1921 Malabar Rebellion and the social struggles in Travancore. In the Malabar Rebellion, Mappila Muslims of Malabar rioted against Hindu zamindars and the British Raj.[107] Some social struggles against caste inequalities also erupted in the early decades of 20th century, leading to the 1936 Temple Entry Proclamation that opened Hindu temples in Travancore to all castes.[108]  After India was partitioned in 1947 into India and Pakistan, Travancore and Kochi, part of the Union of India were merged on 1 July 1949 to form Travancore-Cochin.[109] On 1 November 1956, the taluk of Kasargod in the South Kanara district of Madras, the Malabar district of Madras, and Travancore-Cochin, without four southern taluks (which joined Tamil Nadu), merged to form the state of Kerala under the States Reorganisation Act.[110][111] A Communist-led government under E. M. S. Namboodiripad resulted from the first elections for the new Kerala Legislative Assembly in 1957.[111] It was one of the earliest elected Communist governments, after Communist success in the 1945 elections in the Republic of San Marino.[112][113][114] His government helped distribute land and implement educational reforms.[115]  The state is wedged between the Lakshadweep Sea and the Western Ghats. Lying between northern latitudes 8°18\' and 12°48\' and eastern longitudes 74°52\' and 77°22\',[116] Kerala experiences humid tropical rainforest climate with some cyclones. The state has a coast of 590 km (370 mi)[117] and the width of the state varies between 11 and 121 kilometres (7 and 75 mi).[118] Geographically, Kerala can be divided into three climatically distinct regions: the eastern highlands; rugged and cool mountainous terrain, the central mid-lands; rolling hills, and the western lowlands; coastal plains.[45]:110 Pre-Cambrian and Pleistocene geological formations compose the bulk of Kerala\'s terrain.[119][120] A catastrophic flood in Kerala in 1341 CE drastically modified its terrain and consequently affected its history; it also created a natural harbour for spice transport.[121] The eastern region of Kerala consists of high mountains, gorges and deep-cut valleys immediately west of the Western Ghats\' rain shadow.[45]:110 41 of Kerala\'s west-flowing rivers,[122] and 3 of its east-flowing ones originate in this region.[123][124] The Western Ghats form a wall of mountains interrupted only near Palakkad; hence also known Palghat, where the Palakkad Gap breaks.[125] The Western Ghats rise on average to 1,500 metres (4,900 feet) above sea level,[126] while the highest peaks reach around 2,500 metres (8,200 feet).[127] Anamudi in the Idukki district is the highest peak in south India, is at an elevation of 2,695 m (8,842 ft).[128] The Western Ghats mountain chain is recognised as one of the world\'s eight "hottest hotspots" of biological diversity and is listed among UNESCO World Heritage Sites.[129] The chain\'s forests are considered to be older than the Himalaya mountains.[129]  Kerala\'s western coastal belt is relatively flat compared to the eastern region,[45]:33 and is criss-crossed by a network of interconnected brackish canals, lakes, estuaries,[130] and rivers known as the Kerala Backwaters.[131] The state\'s largest lake Vembanad, dominates the backwaters; it lies between Alappuzha and Kochi and is about 200 km2 (77 sq mi) in area.[132] Around eight percent of India\'s waterways are found in Kerala.[133] Kerala\'s 44 rivers include the Periyar; 244 kilometres (152 mi), Bharathapuzha; 209 kilometres (130 mi), Pamba; 176 kilometres (109 mi), Chaliyar; 169 kilometres (105 mi), Kadalundipuzha; 130 kilometres (81 mi), Chalakudipuzha; 130 kilometres (81 mi), Valapattanam; 129 kilometres (80 mi) and the Achankovil River; 128 kilometres (80 mi). The average length of the rivers is 64 kilometres (40 mi). Many of the rivers are small and entirely fed by monsoon rain.[134] As Kerala\'s rivers are small and lacking in delta, they are more prone to environmental effects. The rivers face problems such as sand mining and pollution.[135] The state experiences several natural hazards like landslides, floods and droughts. The state was also affected by the 2004 Indian Ocean tsunami,[136] and in 2018 received the worst flooding in nearly a century.[137]  With around 120–140 rainy days per year,[138]:80 Kerala has a wet and maritime tropical climate influenced by the seasonal heavy rains of the southwest summer monsoon and northeast winter monsoon.[139] Around 65% of the rainfall occurs from June to August corresponding to the Southwest monsoon, and the rest from September to December corresponding to Northeast monsoon.[139] The moisture-laden winds of the Southwest monsoon, on reaching the southernmost point of the Indian Peninsula, because of its topography, divides into two branches; the "Arabian Sea Branch" and the "Bay of Bengal Branch".[140] The "Arabian Sea Branch" of the Southwest monsoon first hits the Western Ghats,[141] making Kerala the first state in India to receive rain from the Southwest monsoon.[142][143] The distribution of pressure patterns is reversed in the Northeast monsoon, during this season the cold winds from North India pick up moisture from the Bay of Bengal and precipitate it on the east coast of peninsular India.[144][145] In Kerala, the influence of the Northeast monsoon is seen in southern districts only.[146] Kerala\'s rainfall averages 2,923 mm (115 in) annually.[147] Some of Kerala\'s drier lowland regions average only 1,250 mm (49 in); the mountains of the eastern Idukki district receive more than 5,000 mm (197 in) of orographic precipitation: the highest in the state. In eastern Kerala, a drier tropical wet and dry climate prevails. During the summer, the state is prone to gale-force winds, storm surges, cyclone-related torrential downpours, occasional droughts, and rises in sea level.[148]:26, 46, 52 The mean daily temperature ranges from 19.8 °C to 36.7 °C.[149] Mean annual temperatures range from 25.0–27.5 °C in the coastal lowlands to 20.0–22.5 °C in the eastern highlands.[148]:65  Most of the biodiversity is concentrated and protected in the Western Ghats. Three quarters of the land area of Kerala was under thick forest up to 18th century.[150] As of 2004, over 25% of India\'s 15,000 plant species are in Kerala. Out of the 4,000 flowering plant species; 1,272 of which are endemic to Kerala, 900 are medicinal, and 159 are threatened.[151]:11 Its 9,400 km2 of forests include tropical wet evergreen and semi-evergreen forests (lower and middle elevations—3,470 km2), tropical moist and dry deciduous forests (mid-elevations—4,100 km2 and 100 km2, respectively), and montane subtropical and temperate (shola) forests (highest elevations—100 km2). Altogether, 24% of Kerala is forested.[151]:12 Three of the world\'s Ramsar Convention listed wetlands—Lake Sasthamkotta, Ashtamudi Lake and the Vembanad-Kol wetlands—are in Kerala, as well as 1455.4 km2 of the vast Nilgiri Biosphere Reserve. Subjected to extensive clearing for cultivation in the 20th century,[152]:6–7 much of the remaining forest cover is now protected from clearfelling.[153] Eastern Kerala\'s windward mountains shelter tropical moist forests and tropical dry forests, which are common in the Western Ghats.[154][155] The world\'s oldest teak plantation \'Conolly\'s Plot\' is in Nilambur.[156]  Kerala\'s fauna are notable for their diversity and high rates of endemism: it includes 118 species of mammals (1 endemic), 500 species of birds, 189 species of freshwater fish, 173 species of reptiles (10 of them endemic), and 151 species of amphibians (36 endemic).[157] These are threatened by extensive habitat destruction, including soil erosion, landslides, salinisation, and resource extraction. In the forests, sonokeling, Dalbergia latifolia, anjili, mullumurikku, Erythrina, and Cassia number among the more than 1,000 species of trees in Kerala. Other plants include bamboo, wild black pepper, wild cardamom, the calamus rattan palm, and aromatic vetiver grass, Vetiveria zizanioides.[151]:12 Indian elephant, Bengal tiger, Indian leopard, Nilgiri tahr, common palm civet, and grizzled giant squirrels are also found in the forests.[151]:12, 174–175 Reptiles include the king cobra, viper, python, and mugger crocodile. Kerala\'s birds include the Malabar trogon, the great hornbill, Kerala laughingthrush, darter and southern hill myna. In the lakes, wetlands, and waterways, fish such as Kadu, Red Line Torpedo Barb and choottachi; orange chromide—Etroplus maculatus are found.[158][151]:163–165  The state\'s 14 districts are distributed among six regions: North Malabar (far-north Kerala), South Malabar (northern Kerala), Kochi (central Kerala), Northern Travancore, Central Travancore (southern Kerala) and Southern Travancore (far-south Kerala). The districts which serve as administrative regions for taxation purposes are further subdivided into 75 taluks, which have fiscal and administrative powers over settlements within their borders, including maintenance of local land records. Kerala\'s taluks are further sub-divided into 1,453 revenue villages.[159] Since the 73rd and 74th amendments to the Constitution of India, the local government institutions function as the third tier of government, which constitutes 14 District Panchayats, 152 Block Panchayats, 978 Grama Panchayats, 60 Municipalities, six Corporations and one Township.[160] Mahé, a part of the Indian union territory of Puducherry,[161] though 647 kilometres (402 mi) away from it,[162] is a coastal exclave surrounded by Kerala on all of its landward approaches. The Kannur District surrounds Mahé on three sides with the Kozhikode District on the fourth.[163]  There are six Municipal corporations in Kerala that govern Thiruvananthapuram, Kollam, Kochi, Thrissur, Kozhikode and Kannur.[164] The Thiruvananthapuram Municipal Corporation is the largest corporation in Kerala while Kochi metropolitan area named Kochi UA is the largest urban agglomeration.[165] According to a survey by economics research firm Indicus Analytics in 2007, Thiruvananthapuram, Kollam, Kozhikode, Thrissur, Kochi are among the "best cities in India to live"; the survey used parameters such as health, education, environment, safety, public facilities and entertainment to rank the cities.[166]  Kerala hosts two major political alliances: the United Democratic Front (UDF), led by the Indian National Congress; and the Left Democratic Front (LDF), led by the Communist Party of India (Marxist) (CPI(M)). As of 2016, the LDF is the ruling coalition; Pinarayi Vijayan of the Communist Party of India (Marxist) is the Chief Minister, while Ramesh Chennithala of the UDF is the Leader of Opposition. According to the Constitution of India, Kerala has a parliamentary system of representative democracy; universal suffrage is granted to residents.[167] The government is organised into the three branches:  Legislature: The unicameral legislature, the Kerala Legislative Assembly popularly known as Niyamasabha, comprises elected members and special office bearers; the Speaker and Deputy Speaker elected by the members from among themselves. Assembly meetings are presided over by the Speaker and in the Speaker\'s absence, by the Deputy Speaker. The state has 140 assembly constituencies.[168] The state elects 20 and 9 members for representation in the Lok Sabha and the Rajya Sabha respectively.[169] Executive: The Governor of Kerala is the constitutional head of state, and is appointed by the President of India.[170] Arif Mohammad Khan is the Governor of Kerala.[171] The executive authority is headed by the Chief Minister of Kerala, who is the head of government and is vested with extensive executive powers; the head of the majority party in the Legislative Assembly is appointed to the post by the Governor.[170] The Council of Ministers has its members appointed by the Governor, taking the advice of the Chief Minister.[170] The executive administration is based in Thiruvananthapuram at State Secretariat complex. Each district has a district administrator appointed by government called District collector for executive administration. Auxiliary authorities known as panchayats, for which local body elections are regularly held, govern local affairs.[172] Judiciary: The judiciary consists of the Kerala High Court and a system of lower courts.[173] The High Court, located in Kochi,[174] has a Chief Justice along with 23 permanent and seven additional pro tempore justices as of 2012.[175] The high court also hears cases from the Union Territory of Lakshadweep.[176][177] The local government bodies; Panchayat, Municipalities and Corporations have existed in Kerala since 1959, however, the major initiative to decentralise the governance was started in 1993, conforming to the constitutional amendments of central government in this direction.[178] With the enactment of Kerala Panchayati Raj Act and Kerala Municipality Act in 1994, the state implemented reforms in local self-governance.[179] The Kerala Panchayati Raj Act envisages a 3-tier system of local government with Gram panchayat, Block panchayat and District Panchayat forming a hierarchy.[180] The acts ensure a clear demarcation of power among these institutions.[178] However, the Kerala Municipality Act envisages a single-tier system for urban areas, with the institution of municipality designed to par with the Gram panchayat of the former system. Substantial administrative, legal and financial powers are delegated to these bodies to ensure efficient decentralisation.[181] As per the present norms, the state government devolves about 40 per cent of the state plan outlay to the local government.[182]  '

In [15]:
print(context)

Kerala is a state on the southwestern Malabar Coast of India. It was formed on 1 November 1956, following the passage of the States Reorganisation Act, by combining Malayalam-speaking regions of the erstwhile states of Travancore-Cochin and Madras. Spread over 38,863 km2 (15,005 sq mi), Kerala is the twenty-third largest Indian state by area. It is bordered by Karnataka to the north and northeast, Tamil Nadu to the east and south, and the Lakshadweep Sea[11] to the west. With 33,387,677 inhabitants as per the 2011 Census, Kerala is the thirteenth-largest Indian state by population. It is divided into 14 districts with the capital being Thiruvananthapuram. Malayalam is the most widely spoken language and is also the official language of the state.[12]  The Chera Dynasty was the first prominent kingdom based in Kerala. The Ay kingdom in the deep south and the Ezhimala kingdom in the north formed the other kingdoms in the early years of the Common Era (CE or AD). The region had been a pro

In [18]:
print(answer_question("What is Kerala located?", context))

southwestern malabar coast of india
